In [1]:
import pandas as pd
from google.cloud import tpu_v2
from datetime import datetime, timezone

In [2]:
def redable_time(t):
    # ≥ 1 day
    if t >= 86400:  
        days = int(t // 86400)
        return f"{days} day{'s' if days > 1 else ''}"
    # ≥ 1 hour
    elif t >= 3600:
        hours = int(t // 3600)
        return f"{hours} hour{'s' if hours > 1 else ''}"
    # minutes
    else:
        minutes = max(1, int(t // 60))  # At least 1 minute
        return f"{minutes} minute{'s' if minutes > 1 else ''}"

In [3]:
project_id = 'personal-project-451418'

In [4]:
# list all TPUs across all zones
client = tpu_v2.TpuClient()
parent = f'projects/{project_id}/locations/-'
nodes = client.list_nodes(parent=parent)

# create table
rows = []
for node in nodes:
    row = {
        'id': node.name.split('/')[-1],
        'zone': node.name.split('/')[3],
        'type': node.accelerator_type,
        'state': node.state.name,
        'health': node.health.name,
        'uptime': (datetime.now(timezone.utc) - node.create_time).total_seconds(),
        # 'health desc.': node.health_description,
    }
    rows += [row]
df = pd.DataFrame(rows)
df.sort_values(by='uptime', ascending=False, inplace=True)

# color rows
def highlight_rows(row):
    color_map = {
        'READY': 'background-color: #66cc99',
        'CREATING': 'background-color: #fffc7f',
        'DELETING': 'background-color: #fe7968',
    }
    return [color_map.get(row['state'], '') for _ in row]
df.style.apply(highlight_rows, axis=1).format({
  'uptime': redable_time,
})

,id,zone,type,state,health,uptime
6,tpu-v4-64,us-central2-b,v4-64,READY,HEALTHY,7 days
3,sweep-1-2,europe-west4-a,v3-8,DELETING,HEALTHY,8 minutes
4,sweep-1-3,europe-west4-a,v3-8,DELETING,HEALTHY,7 minutes
1,sweep-1-4,europe-west4-a,v3-8,DELETING,HEALTHY,6 minutes
2,sweep-1-5,europe-west4-a,v3-8,DELETING,HEALTHY,5 minutes
0,sweep-1-6,europe-west4-a,v3-8,DELETING,HEALTHY,4 minutes
5,sweep-1-1,europe-west4-a,v3-8,READY,HEALTHY,4 minutes
